### Notebook qui reprend GTD_preprocessing.ipynb
GTD_preprocessing
Fait a la zeub parce que l'api est pas tres stable, permet de faire des request moreceau par morceaux

In [1]:
import pandas as pd

# External librairies
# Google trend unofficial API
from pytrends.request import TrendReq
from datetime import datetime


data_folder = './Data/'

In [12]:
pytrends = TrendReq(hl='fr-FR',tz=60, retries=10,backoff_factor=0.1,)


def extract(dict_in, list_out, list_out2):
    '''
    extract all integers conatined in a nested dictionary with list
    code adapted from here : https://stackoverflow.com/questions/10569636/how-to-get-all-keysvalues-in-nested-dict-of-list-of-dicts-and-dicts/10569687
    '''
    for key, value in dict_in.items():
        if isinstance(value, dict): # If value itself is dictionary
            extract(value, list_out, list_out2)
        elif isinstance(value, list):
            for key2 in value:
                extract(key2, list_out, list_out2)
            # Write to list_out
        elif isinstance(value, int):
            list_out.append(value)
        elif key=='name':
            list_out2.append(value) 
    return list_out,list_out2

categories_dictionary = pytrends.categories()
categories_ids =[]
categories_names = []
[categories_ids,categories_names] = extract(categories_dictionary,categories_ids,categories_names)
categories_df = pd.DataFrame(zip(categories_ids,categories_names),columns=['id','name']).sort_values('id')


categories_df.to_csv(data_folder + 'categories.csv')
categories_df=categories_df.drop_duplicates(subset ="id")
categories_df

,id,name
1426,0,Toutes catégories
240,3,Arts et divertissements
715,5,Informatique et électronique
396,7,Finance
785,8,Jeux
...,...,...
612,1393,Décodeurs et magnétoscopes numériques
614,1394,Lecteurs et enregistreurs Blu-Ray
615,1395,Lecteurs et enregistreurs DVD
603,1396,Oreillettes


In [13]:
categories_df = categories_df.iloc[j:,:]
categories_df

,id,name
858,271,Appareils ménagers
880,272,Lampes et luminaires
301,273,Motos
336,275,Partage de photos et vidéos
51,276,Restaurants
...,...,...
612,1393,Décodeurs et magnétoscopes numériques
614,1394,Lecteurs et enregistreurs Blu-Ray
615,1395,Lecteurs et enregistreurs DVD
603,1396,Oreillettes


In [14]:

GTD = pd.DataFrame()
for j,i in enumerate(categories_df['id']):
    kw_list = [""]
    pytrends.build_payload(kw_list, geo='FR', cat=i , timeframe='2004-01-01 2020-12-31')

    temp = pytrends.interest_over_time()
    if not temp.empty:
        GTD[i]=temp.iloc[:,0]
    
    if i/70==i//70:
        print(i/1427)    
    
GTD

0.19621583742116327
0.2452697967764541
0.29432375613174494
0.39243167484232655
0.4414856341976174
0.4905395935529082
0.5886475122634899
0.6377014716187807
0.6867554309740714
0.7358093903293623
0.7848633496846531
0.833917309039944
0.8829712683952348
0.9320252277505255


,271,272,273,275,276,277,278,279,284,287,...,1381,1385,1386,1387,1388,1389,1390,1391,1392,1397
date,,,,,,,,,,,,,,,,,,,,,
2004-01-01,46,49,46,19,44,55,26,60,48,79,...,4,67,41,19,29,30,58,84,73,31
2004-02-01,44,43,52,19,50,51,37,63,50,77,...,4,100,52,38,72,60,85,81,92,26
2004-03-01,38,40,63,19,42,57,67,67,48,85,...,2,51,51,21,66,37,39,82,71,26
2004-04-01,41,42,72,20,46,54,42,71,48,86,...,3,88,89,33,76,75,56,78,98,26
2004-05-01,40,41,73,22,44,52,20,56,52,80,...,3,14,51,23,45,59,100,70,78,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-01,72,64,40,16,100,91,19,43,59,65,...,31,9,12,9,23,34,79,50,20,9
2020-09-01,63,66,38,17,79,81,29,48,61,70,...,31,14,24,9,32,24,76,42,11,9
2020-10-01,73,77,33,16,77,76,36,47,69,75,...,34,11,19,11,33,18,78,43,9,10


In [9]:
i

271

In [15]:
# datetime object containing current date and time
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d_%H-%M-%S")
GTD.to_csv(data_folder+ dt_string + 'GTD.csv')